<img src="./images/logo.png" alt="Drawing" style="width: 500px;"/>

# Exercise 5: Track your Experiment and Register the Model

## What we will cover:
- Find saved artifacts & metrics on MLflow
- Register the model and move it to Production
- Prediction test to validate the results

## 1. Validate that the Model was generated and Stored in MLflow

### Head back to the menu and select `Experiments`
![image.png](./images/exercise5/menu2.PNG)

### It will open MLflow, select your experiment (e.g. group01-end2end-retail-demo)
- You can see here a list of the runs you've done
- On the right clic on `show more columns` then you can compare the metrics stored in the model like `accuracy` and `loss`
![image.png](./images/exercise5/mlflow.PNG)

### Click on a specific run name to see more details
- `parameters`: Displays all the parameters used and logged in the model
- `metrics`: Displays all the metric used and logged in the model
- `Artifacts`: Displays all the artifacts logged in the model
- `Full path`: S3 URI to access the model

![image.png](./images/exercise5/model.PNG)


- `Predict on a Pandas DataFrame:` at the bottom
Look how it is used, it might be relevant for later

![image.png](./images/exercise5/predict_from_mlflow.PNG)

### Set up the group parameter

In [ ]:
## set your group name (in quotes)
group_name = ""

In [ ]:
# Check if GROUP is empty
if not group_name:
    print("Please set your group name before proceeding to the next cell.")
    # You can also raise an exception to stop execution if needed
    # raise ValueError("Group name is empty. Please set a valid group name.")
else:
    print("Group name is set. Proceed to the next cell.")
    # Your code for the next cell can go here

### Double Checking

In [ ]:
# Check if GROUP is empty
if not group_name:
    print("Please set your group name before proceeding to the next cell.")
    # You can also raise an exception to stop execution if needed
    # raise ValueError("Group name is empty. Please set a valid group name.")

# Prerequisites 
<div class="alert alert-block alert-danger">
<b>Important</b> Make sure it's valid
</div>


In [ ]:
# Set parametes

# adapt to your EZUA Domain name
EZAF_ENV = "pe1.ezmeral.de"
# type of demo (DemoContent - in this case fruit and vegetables) 
DC = "fruit"
# path to end2end demo (not data)
end2end_path = '/mnt/datasources/datafabric/ezua/end2end/' 
# path to data for model training, etc.
path = '/mnt/datasources/datafabric/ezua/end2end-data/fruits/' 
# path to GROUP INDIVIDUAL data for model training, etc.
group_data_path = '/mnt/datasources/datafabric/ezua/end2end-group-data/' 
# experiment name prefix for mlflow
experiment_name = "end2end-retail-demo"
exp_name = group_name + "-" + experiment_name
model_name = "end2end-retail-demo"
g_model_name = group_name + "-" + model_name
# artifact_path = "end2end-retail-demo"
artifact_path = "model"
# password for UA user login (needed to get keycloak token)
UA_password = "Hpepoc@123"

### Import required libraries & refresh the token
- Ignore the warnings

In [ ]:
import mlflow
from mlflow.tracking.client import MlflowClient
from mlflow.entities.model_registry.model_version_status import ModelVersionStatus
from IPython.display import display
from PIL import Image
import numpy as np
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from io import BytesIO

In [ ]:
%update_token

## 2. Register the Model<a class="anchor" id="6"></a>

### Search for the last run ID for your Group

In [ ]:
# Search for runs in the specified experiment, ordering by start time in descending order
runs = mlflow.search_runs(experiment_ids=[mlflow.get_experiment_by_name(exp_name).experiment_id],
                          order_by=["start_time desc"],
                          filter_string="")

# Check if there are any runs
if not runs.empty:
    # Get the run ID of the last active run
    last_run_id = runs.iloc[0]["run_id"]
    print("Last active run ID for experiment '{}': {}".format(exp_name, last_run_id))
else:
    print("No runs found for experiment '{}'.".format(exp_name))

### Create a new model version (Version 1 , Version2, ...) in MLflow model registry:
- Form the model_uri specifying the location of model files using the run_id and artifact_path.
- Register the model in MLflow with the specified model_uri and g_model_name.
- Initiate the creation of a new model version and waits for it to become ready.
- Check and prints the status of the model version until it becomes "READY".
- Update the description of the registered model.

In [ ]:
# set parameters model_uri and model details and register model in mlflow
model_uri = "runs:/{run_id}/{artifact_path}".format(run_id=last_run_id, artifact_path=artifact_path)
model_details = mlflow.register_model(model_uri=model_uri, name=g_model_name)

# Wait until the model is ready
def wait_until_ready(g_model_name, model_version):
    client = MlflowClient()
    for _ in range(10):
        model_version_details = client.get_model_version(
          name=g_model_name,
          version=model_version,
        )
        status = ModelVersionStatus.from_string(model_version_details.status)
        print("Model status: %s" % ModelVersionStatus.to_string(status))
        if status == ModelVersionStatus.READY:
            break
        time.sleep(1)
        client.update_registered_model(
            name=model_details.name,
            description="Fruit & Vegetables Cashierless Store"
        )

wait_until_ready(model_details.name, model_details.version)

### Transition the model to `Production` stage

In [ ]:
# create an instance of the MlflowClient
client = MlflowClient()

# Get the latest model created for our experiment
latest_versions = client.get_latest_versions(name=g_model_name, stages=["None"])
latest_version = latest_versions[0]

# Transition the desired model version to production stage
client.transition_model_version_stage(
  name=g_model_name,
  version=latest_version.version,
  stage='Production',
)

### Transition the other models to `Archived` stage

In [ ]:
# Transition model versions to a different stage if their current stage is not "production"
model_versions = client.search_model_versions("")

# Transition model versions to a different stage if their current stage is not "production"
for mv in model_versions:
    if mv.name == g_model_name:
        if mv.version != latest_version.version:
            client.transition_model_version_stage(
                name=mv.name,
                version=mv.version,
                stage="Archived"
            )
            print(f"Model: {mv.name}, version: {mv.version} has been moved to Archived")

            # Update Model Version Description
            client.update_model_version(
                name=mv.name,
                version=mv.version,
                description="Model Moved to Archived"
            )

### Let's test if it works
- Use the `Predict on a Pandas DataFrame:` that we've seen ealier
- `Fill in` the missing parts

In [ ]:
# Get the source URI or location of the model version
#logged_model = latest_version.source
logged_model = "Fill in"

# Load model as a PyFuncModel.
#loaded_model = mlflow.pyfunc.load_model(logged_model)
loaded_model = "Fill in"

### Similar function as in Exercise4 but we update the prediction to use the MLflow model instead
- `Fill in` the missing parts

In [ ]:
def predict(url):
    # Preprocess the image
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    img = img.resize((224, 224))  
    img_array = img_to_array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Make a prediction
    prediction = loaded_model.predict(img_array)   # We update the previous function to use the loaded model here
    predicted_class = np.argmax(prediction, axis=-1)
    result = labels[predicted_class[0]]
    
    # Display the image
    display(img)

    return result

In [ ]:
# Define your labels
labels = ['apple', 'banana', 'ezmeral', 'kiwi', 'orange']

# Example usage with an image URL
image_url = "Fill in"

predicted_label = predict(image_url)
print("The model predicts: " + str(predicted_label))

## END